In [38]:
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import ComplementNB, MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Embedding, InputLayer, Dropout, BatchNormalization, Flatten

# 실험 설계

**num_words = [10000, 20000, max(30979+3)], with tokens**  
**models = [LogisticRegression, 그래디언트 부스팅 트리, 보팅]**


## 평가 기준

### 1. 분류 모델의 accuracy가 기준 이상 높게 나왔는가?
  * 3가지 단어 개수에 대해 8가지 머신러닝 기법을 적용하여 그중 최적의 솔루션을 도출하였다.
  
### 2. 분류 모델의 F1 score가 기준 이상 높게 나왔는가?
  * Vocabulary size에 따른 각 머신러닝 모델의 성능변화 추이를 살피고, 해당 머신러닝 알고리즘의 특성에 근거해 원인을 분석하였다.  
  
### 3. 딥러닝 모델을 활용해 성능이 비교 및 확인되었는가?
  * 동일한 데이터셋과 전처리 조건으로 딥러닝 모델의 성능과 비교하여 결과에 따른 원인을 분석하였다.  

In [3]:
def graph_confusion_matrix(model, x_test, y_test):#, classes_name):
    df_cm = pd.DataFrame(confusion_matrix(y_test, model.predict(x_test)))#, index=classes_name, columns=classes_name)
    fig = plt.figure(figsize=(12,12))
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=12)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=12)
    plt.ylabel('label')
    plt.xlabel('predicted value')

In [5]:
def load_data(num_words=None):
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words, test_split=0.2)
    word_index = reuters.get_word_index(path="reuters_word_index.json")

    index_to_word = { index+3 : word for word, index in word_index.items() }

    for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
        index_to_word[index]=token

    decoded = []
    for i in range(len(x_train)):
        t = ' '.join([index_to_word[index] for index in x_train[i]])
        decoded.append(t)

    x_train = decoded

    decoded = []
    for i in range(len(x_test)):
        t = ' '.join([index_to_word[index] for index in x_test[i]])
        decoded.append(t)

    x_test = decoded

    dtmvector = CountVectorizer()
    tfidf_transformer = TfidfTransformer()

    # 데이터를 DTM으로 변환 -> DTM을 TF-IDF 행렬로 변환
    x_train_dtm = dtmvector.fit_transform(x_train)
    tfidfv_train = tfidf_transformer.fit_transform(x_train_dtm)

    x_test_dtm = dtmvector.transform(x_test)
    tfidfv_test = tfidf_transformer.transform(x_test_dtm)

    return tfidfv_train, y_train, tfidfv_test, y_test

In [6]:
def run_models(num_words=None):
    x_train, y_train, x_test, y_test = load_data(num_words=num_words)

    # 1.Logistic Regression
    lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000)
    lr.fit(x_train, y_train)

    predicted_lr = lr.predict(x_test)
    print("--"*10 + "Logistic Regression" + "--"*10)
    print("Logistic Regression 정확도:", accuracy_score(y_test, predicted_lr))

    print(classification_report(y_test, predicted_lr, zero_division=0))
    print("---"*10+"\n")

    # 2.Linear SVC
    lsvc = LinearSVC(C=1000, penalty='l1', max_iter=3000, dual=False)
    lsvc.fit(x_train, y_train)

    predicted_lsvc = lsvc.predict(x_test)
    print("--"*10 + "Linear SVC" + "--"*10)
    print("정확도:", accuracy_score(y_test, predicted_lsvc))

    print(classification_report(y_test, predicted_lsvc, zero_division=0))
    print("---"*10+"\n")

    # 3.Voting(Logistic Regression, Complement Naive Bayes Classifier, GradientBoosting Classifier)
    # 각 개별 모델 선언
    logistic_regression = LogisticRegression(penalty='l2', random_state=0)
    complement_nb = ComplementNB()
    gradient_boosting = GradientBoostingClassifier(random_state=0)

    # VotingClassifier 선언
    voting_classifier = VotingClassifier(
        estimators=[
            ('logistic', logistic_regression),
            ('complement_nb', complement_nb),
            ('gradient_boosting', gradient_boosting)
        ],
        voting='soft'
    )

    voting_classifier.fit(x_train, y_train)

    predicted_voting = voting_classifier.predict(x_test)
    print("--"*10 + "Voting Classifier" + "--"*10)
    print("정확도:", accuracy_score(y_test, predicted_voting))

    print(classification_report(y_test, predicted_voting, zero_division=0))
    print("---"*10+"\n")

In [10]:
run_models(num_words=10000)

550378/550378 [==============================] - 0s 0us/step
--------------------Logistic Regression--------------------
Logistic Regression 정확도: 0.8107747105966162
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.74      0.78      0.76       105
           2       0.70      0.70      0.70        20
           3       0.91      0.93      0.92       813
           4       0.80      0.87      0.84       474
           5       1.00      0.20      0.33         5
           6       0.93      0.93      0.93        14
           7       1.00      0.33      0.50         3
           8       0.68      0.71      0.69        38
           9       0.81      0.88      0.85        25
          10       0.93      0.87      0.90        30
          11       0.67      0.72      0.70        83
          12       0.62      0.38      0.48        13
          13       0.65      0.59      0.62        37
          14       0.67 

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


--------------------Linear SVC--------------------
정확도: 0.7845057880676759
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.74      0.74      0.74       105
           2       0.57      0.65      0.60        20
           3       0.91      0.91      0.91       813
           4       0.80      0.84      0.82       474
           5       0.00      0.00      0.00         5
           6       0.86      0.86      0.86        14
           7       0.50      0.33      0.40         3
           8       0.66      0.71      0.68        38
           9       0.77      0.80      0.78        25
          10       0.92      0.77      0.84        30
          11       0.66      0.77      0.71        83
          12       0.38      0.38      0.38        13
          13       0.62      0.57      0.59        37
          14       0.67      1.00      0.80         2
          15       0.62      0.56      0.59         9
      

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


--------------------Voting Classifier--------------------
정확도: 0.7969723953695459
              precision    recall  f1-score   support

           0       0.82      0.75      0.78        12
           1       0.76      0.72      0.74       105
           2       0.78      0.70      0.74        20
           3       0.91      0.93      0.92       813
           4       0.78      0.88      0.82       474
           5       0.00      0.00      0.00         5
           6       0.80      0.86      0.83        14
           7       1.00      0.33      0.50         3
           8       0.69      0.66      0.68        38
           9       0.81      0.84      0.82        25
          10       0.90      0.87      0.88        30
          11       0.66      0.71      0.68        83
          12       0.33      0.46      0.39        13
          13       0.71      0.65      0.68        37
          14       0.14      0.50      0.22         2
          15       0.43      0.33      0.38         9

In [8]:
run_models(num_words=20000)

550378/550378 [==============================] - 0s 0us/step
--------------------Logistic Regression--------------------
Logistic Regression 정확도: 0.8147818343722173
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.73      0.79      0.76       105
           2       0.78      0.70      0.74        20
           3       0.92      0.93      0.92       813
           4       0.80      0.88      0.84       474
           5       1.00      0.20      0.33         5
           6       0.93      0.93      0.93        14
           7       1.00      0.33      0.50         3
           8       0.69      0.71      0.70        38
           9       0.85      0.88      0.86        25
          10       0.96      0.87      0.91        30
          11       0.68      0.72      0.70        83
          12       0.62      0.38      0.48        13
          13       0.65      0.65      0.65        37
          14       0.67 

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


--------------------Linear SVC--------------------
정확도: 0.7934105075690115
              precision    recall  f1-score   support

           0       0.80      0.67      0.73        12
           1       0.74      0.74      0.74       105
           2       0.72      0.65      0.68        20
           3       0.91      0.92      0.92       813
           4       0.82      0.86      0.84       474
           5       0.50      0.20      0.29         5
           6       0.88      1.00      0.93        14
           7       1.00      0.67      0.80         3
           8       0.66      0.71      0.68        38
           9       0.78      0.84      0.81        25
          10       0.88      0.77      0.82        30
          11       0.66      0.73      0.70        83
          12       0.45      0.38      0.42        13
          13       0.59      0.51      0.55        37
          14       0.67      1.00      0.80         2
          15       0.56      0.56      0.56         9
      

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


--------------------Voting Classifier--------------------
정확도: 0.798753339269813
              precision    recall  f1-score   support

           0       0.80      0.67      0.73        12
           1       0.77      0.74      0.76       105
           2       0.68      0.75      0.71        20
           3       0.90      0.93      0.92       813
           4       0.77      0.89      0.83       474
           5       0.50      0.20      0.29         5
           6       0.91      0.71      0.80        14
           7       0.25      0.33      0.29         3
           8       0.70      0.68      0.69        38
           9       0.95      0.80      0.87        25
          10       0.83      0.83      0.83        30
          11       0.69      0.72      0.71        83
          12       0.50      0.46      0.48        13
          13       0.65      0.54      0.59        37
          14       0.14      0.50      0.22         2
          15       0.38      0.33      0.35         9


In [9]:
run_models(num_words=None)

--------------------Logistic Regression--------------------
Logistic Regression 정확도: 0.8161175422974176
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.75      0.79      0.77       105
           2       0.78      0.70      0.74        20
           3       0.92      0.93      0.93       813
           4       0.81      0.88      0.84       474
           5       1.00      0.20      0.33         5
           6       0.93      0.93      0.93        14
           7       1.00      0.33      0.50         3
           8       0.71      0.71      0.71        38
           9       0.85      0.88      0.86        25
          10       0.93      0.90      0.92        30
          11       0.67      0.72      0.70        83
          12       0.57      0.31      0.40        13
          13       0.64      0.62      0.63        37
          14       0.67      1.00      0.80         2
          15       0.80      0.

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


--------------------Linear SVC--------------------
정확도: 0.7916295636687445
              precision    recall  f1-score   support

           0       0.80      0.67      0.73        12
           1       0.71      0.76      0.74       105
           2       0.75      0.75      0.75        20
           3       0.91      0.92      0.91       813
           4       0.81      0.84      0.82       474
           5       0.00      0.00      0.00         5
           6       0.93      0.93      0.93        14
           7       1.00      0.33      0.50         3
           8       0.62      0.68      0.65        38
           9       0.81      0.84      0.82        25
          10       0.92      0.77      0.84        30
          11       0.66      0.73      0.69        83
          12       0.38      0.38      0.38        13
          13       0.59      0.59      0.59        37
          14       0.50      1.00      0.67         2
          15       0.67      0.44      0.53         9
      

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


--------------------Voting Classifier--------------------
정확도: 0.8000890471950134
              precision    recall  f1-score   support

           0       0.64      0.58      0.61        12
           1       0.80      0.78      0.79       105
           2       0.67      0.70      0.68        20
           3       0.91      0.93      0.92       813
           4       0.78      0.89      0.83       474
           5       1.00      0.20      0.33         5
           6       0.77      0.71      0.74        14
           7       1.00      0.33      0.50         3
           8       0.67      0.68      0.68        38
           9       0.95      0.80      0.87        25
          10       0.83      0.80      0.81        30
          11       0.68      0.71      0.69        83
          12       0.50      0.46      0.48        13
          13       0.65      0.59      0.62        37
          14       0.00      0.00      0.00         2
          15       0.25      0.11      0.15         9

## 모델별 결과 비교

| num_words | Model               | Accuracy | Macro avg. F1-score | Weighted avg. F1-score |
|-----------|---------------------|----------|---------------------|-----------------------|
| 10000     | Logistic Regression |    0.81     |    0.67                |   0.81                   |
|           | Linear SVC          |    0.78     |    0.64                |   0.78                   |
|           | Voting Classifier   |    0.80     |    0.60                |   0.79                   |
|-----------|---------------------|----------|---------------------|-----------------------|
| 20000     | Logistic Regression |    0.81     |    0.67                |   0.81                   |
|           | Linear SVC          |    0.79     |    0.66                |   0.79                   |
|           | Voting Classifier   |    0.80     |    0.60                |   0.79                   |
|-----------|---------------------|----------|---------------------|-----------------------|
| 30000     | Logistic Regression |    0.82     |    0.67                |   0.81                   |
|           | Linear SVC          |    0.79     |    0.63                |   0.79                   |
|           | Voting Classifier   |    0.80     |    0.60                |   0.79                   |


## 딥러닝 실험

In [34]:
x_train, y_train, x_test, y_test = load_data(num_words=10000)

# print(x_train.shape, x_test.shape)
# (8982, 9670) (2246, 9670)

# TF-IDF 행렬을 3D로 변환합니다.
x_train = np.expand_dims(x_train.toarray(), axis=-1)
x_test = np.expand_dims(x_test.toarray(), axis=-1)

# print(x_train.shape, x_test.shape)
# (8982, 9670, 1) (2246, 9670, 1)

# Conv1D 기반 Keras 모델을 생성합니다.
model = Sequential()
model.add(InputLayer(input_shape=(x_train.shape[1], 1)))
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))  # 드롭아웃 추가
model.add(Dense(46, activation='softmax'))  # 46개의 클래스

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

# 모델 훈련
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 모델 평가
predicted = model.predict(x_test)
predicted = np.argmax(predicted, axis=1)
print("정확도:", accuracy_score(y_test, predicted))
print(classification_report(y_test, predicted, zero_division=0))


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 9666, 128)         768       
                                                                 
 global_max_pooling1d_5 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 46)                5934      
                                                                 
Total params: 6702 (26.18 KB)
Trainable params: 6702 (26.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
(8982, 9670, 1) (8982,) (2246, 9670, 1) (2246,)
Epoch 1/10
281/281 [=================

KeyboardInterrupt: 

In [37]:
# Conv1D 기반 Keras 모델을 생성합니다.
model = Sequential()
model.add(InputLayer(input_shape=(x_train.shape[1], 1)))
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(46, activation='softmax'))  # 46개의 클래스

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

# 모델 훈련
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 모델 평가
predicted = model.predict(x_test)
predicted = np.argmax(predicted, axis=1)
print("정확도:", accuracy_score(y_test, predicted))
print(classification_report(y_test, predicted, zero_division=0))

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 9666, 128)         768       
                                                                 
 global_max_pooling1d_7 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_8 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 46)                5934      
                                                     

KeyboardInterrupt: 

In [46]:
model = Sequential()
model.add(InputLayer(input_shape=(x_train.shape[1], 1)))
model.add(Conv1D (kernel_size=60, filters=128, strides=3, padding='valid', activation='relu'))
model.add(MaxPooling1D(1, padding = 'valid'))

model.add(Conv1D(kernel_size=60, filters=64, activation='relu'))
model.add(MaxPooling1D(1, padding = 'valid'))

model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(46, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 3204, 128)         7808      
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 3204, 128)         0         
 g1D)                                                            
                                                                 
 conv1d_19 (Conv1D)          (None, 3145, 64)          491584    
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 3145, 64)          0         
 ng1D)                                                           
                                                                 
 batch_normalization_11 (Ba  (None, 3145, 64)          256       
 tchNormalization)                                               
                                                     

In [47]:
# 모델 훈련
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 모델 평가
predicted = model.predict(x_test)
predicted = np.argmax(predicted, axis=1)
print("정확도:", accuracy_score(y_test, predicted))
print(classification_report(y_test, predicted, zero_division=0))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-

In [51]:
# 왜 에러가 나는지..? 그냥 모델 구조만 조금 바꾼건데